In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv("LifeExpectancyDataset - Sheet1.csv")
print(df.shape)

df = df.dropna() 
print(df.shape)

(2938, 22)
(1649, 22)


In [2]:
df.loc[df["Status"] == "Developing", "Status"] = 0
df.loc[df["Status"] == "Developed", "Status"] = 1


train_proportion = 0.8

def stratified_split(df, group_col, train_proportion):
    train_df = df.groupby(group_col, group_keys=False).apply(lambda x: x.sample(frac=train_proportion))
    test_df = df.drop(train_df.index)
    return train_df, test_df

train_df, test_df = stratified_split(df, 'Country', train_proportion)

train_df = pd.get_dummies(train_df, columns=["Country"])
test_df = pd.get_dummies(test_df, columns=["Country"])
train_df, test_df = train_df.align(test_df, join='outer', axis=1, fill_value=0)

train_df = train_df.astype(float)
test_df = test_df.astype(float)

train_labels = train_df["Life expectancy"]
train_data = train_df.drop(columns=["Life expectancy"])

test_labels = test_df["Life expectancy"]
test_data = test_df.drop(columns=["Life expectancy"])

print(train_df.head(50))

     Adult Mortality  Alcohol   BMI  Country_Afghanistan  Country_Albania  \
518            281.0     0.01  16.2                  1.0              0.0   
516            275.0     0.01  17.2                  1.0              0.0   
515            272.0     0.01  17.6                  1.0              0.0   
514            268.0     0.01  18.1                  1.0              0.0   
521            295.0     0.03  14.7                  1.0              0.0   
524            295.0     0.01  13.4                  1.0              0.0   
519            287.0     0.03  15.7                  1.0              0.0   
520            295.0     0.02  15.2                  1.0              0.0   
512            263.0     0.01  19.1                  1.0              0.0   
522            291.0     0.02  14.2                  1.0              0.0   
526            316.0     0.01  12.6                  1.0              0.0   
517            279.0     0.01  16.7                  1.0              0.0   

In [3]:
print(train_labels.shape)
print(test_data.shape)
print(train_data)
print(test_labels.shape)
print(train_data.columns)
print(test_data.columns)

(1319,)
(330, 153)
      Adult Mortality  Alcohol   BMI  Country_Afghanistan  Country_Albania  \
518             281.0     0.01  16.2                  1.0              0.0   
516             275.0     0.01  17.2                  1.0              0.0   
515             272.0     0.01  17.6                  1.0              0.0   
514             268.0     0.01  18.1                  1.0              0.0   
521             295.0     0.03  14.7                  1.0              0.0   
...               ...      ...   ...                  ...              ...   
2930             67.0     3.88  28.2                  0.0              0.0   
2937            665.0     1.68  25.5                  0.0              0.0   
2934            715.0     4.06  26.7                  0.0              0.0   
2926            464.0     6.00  29.9                  0.0              0.0   
2932            717.0     4.14  27.5                  0.0              0.0   

      Country_Algeria  Country_Angola  Count

In [4]:
## now ready to build the model
class Linear_model:
    def __init__(self, data, labels):
        # expecting all of them to be numpy ndarray
        self.data = data.to_numpy()
        self.labels = np.array(labels)
        self.w = np.zeros(np.shape(data)[1]).astype(float)
        self.b = 0.0
    
    def normalize(self, data, training=False):
        if training:
            self.mean_val = np.mean(data, axis=0)
            self.std_val = np.std(data, axis=0)
            
#         if data.ndim == 1:
#             data = data.reshape(1, len(data))
        data = (data - self.mean_val) / self.std_val
        return data
    
    def train(self, epochs, a):
        self.data = self.normalize(self.data, True)
        for i in range(epochs):
            self.update_w_and_b(a)
            print("loss: ", self.calc_loss())
            
    def calc_loss(self):
        w = np.copy(self.w) # order feat,
        b = self.b # a constant value
        labels = np.copy(self.labels) # order n,
        data = np.copy(self.data) # order n*p
        n = len(data)
        loss_arr = (labels - (np.sum(w*data, axis=1) + b))**2
        loss = np.sum(loss_arr)/n
        return loss
        
    def update_w_and_b(self, a):
        data = self.data
        w = self.w
        b = self.b
        labels = self.labels
        n = len(data)
        
        predictions = np.dot(data, w) + b
        diff = labels - predictions
        
        grad_w = -2/n * np.dot(data.T, diff)
        grad_b = -2/n * np.sum(diff)
        
        self.w -= a * grad_w
        self.b -= a * grad_b
        
    def predict(self, X):
        X = X.to_numpy()
        X = self.normalize(X)
        return np.dot(X, self.w) + self.b
    
    def evaluate(self, X, y):
        n = len(X)
        predictions = self.predict(X)
        loss_arr = (y - predictions)**2
        loss = np.sqrt(np.sum(loss_arr)/n)
        return loss

In [5]:
# now test run was successful

a = 0.01
model2 = Linear_model(train_data, train_labels)
model2.train(epochs=10000, a=a)

loss:  4674.6414643747885
loss:  4481.375885942829
loss:  4297.700076113969
loss:  4122.760426602369
loss:  3955.855771635278
loss:  3796.4013282193805
loss:  3643.9017012264476
loss:  3497.9306467162583
loss:  3358.1158756313203
loss:  3224.1276182889405
loss:  3095.669996417493
loss:  2972.474492457284
loss:  2854.2949867753637
loss:  2740.9039681843187
loss:  2632.0896235097484
loss:  2527.6535867048233
loss:  2427.4091837023802
loss:  2331.1800506933882
loss:  2238.7990344490427
loss:  2150.1073063601084
loss:  2064.9536390599587
loss:  1983.1938073224928
loss:  1904.6900844963286
loss:  1829.310812881608
loss:  1756.9300317929192
loss:  1687.427151041274
loss:  1620.6866605523742
loss:  1556.5978690728969
loss:  1495.0546665913846
loss:  1435.9553063573853
loss:  1379.2022033274015
loss:  1324.7017465777933
loss:  1272.3641237618463
loss:  1222.103156094592
loss:  1173.8361426574288
loss:  1127.4837130495546
loss:  1082.9696875930163
loss:  1040.2209444363828
loss:  999.1672930089

loss:  2.8034099781114747
loss:  2.802557265489861
loss:  2.801708887052356
loss:  2.8008648074551763
loss:  2.800024991815609
loss:  2.799189405701521
loss:  2.7983580151211793
loss:  2.7975307865134256
loss:  2.7967076867381757
loss:  2.7958886830672145
loss:  2.795073743175296
loss:  2.794262835131527
loss:  2.7934559273910278
loss:  2.7926529887868607
loss:  2.791853988522209
loss:  2.7910588961627973
loss:  2.7902676816295697
loss:  2.7894803151915673
loss:  2.788696767459047
loss:  2.787917009376799
loss:  2.7871410122176785
loss:  2.7863687475763235
loss:  2.785600187363065
loss:  2.784835303798023
loss:  2.7840740694053725
loss:  2.7833164570077815
loss:  2.7825624397210103
loss:  2.781811990948669
loss:  2.7810650843771247
loss:  2.7803216939705555
loss:  2.779581793966143
loss:  2.7788453588694026
loss:  2.778112363449647
loss:  2.777382782735566
loss:  2.7766565920109336
loss:  2.775933766810445
loss:  2.7752142829156345
loss:  2.7744981163509377
loss:  2.773785243379835
los

loss:  2.6139033170511086
loss:  2.6137164863907048
loss:  2.613530038551554
loss:  2.6133439723202843
loss:  2.6131582864890324
loss:  2.612972979855423
loss:  2.612788051222515
loss:  2.6126034993987877
loss:  2.612419323198091
loss:  2.6122355214396236
loss:  2.612052092947897
loss:  2.6118690365526986
loss:  2.6116863510890624
loss:  2.611504035397239
loss:  2.6113220883226593
loss:  2.6111405087159096
loss:  2.6109592954326892
loss:  2.6107784473337867
loss:  2.6105979632850524
loss:  2.6104178421573585
loss:  2.6102380828265748
loss:  2.610058684173537
loss:  2.6098796450840127
loss:  2.6097009644486797
loss:  2.609522641163091
loss:  2.609344674127643
loss:  2.609167062247552
loss:  2.608989804432823
loss:  2.6088128995982185
loss:  2.608636346663235
loss:  2.6084601445520703
loss:  2.608284292193595
loss:  2.6081087885213305
loss:  2.6079336324734155
loss:  2.6077588229925794
loss:  2.6075843590261156
loss:  2.6074102395258585
loss:  2.6072364634481477
loss:  2.607063029753811


loss:  2.5614433458806536
loss:  2.56134801707348
loss:  2.5612528258912897
loss:  2.561157772029894
loss:  2.5610628551859755
loss:  2.560968075057084
loss:  2.560873431341635
loss:  2.560778923738903
loss:  2.5606845519490236
loss:  2.560590315672985
loss:  2.56049621461263
loss:  2.56040224847065
loss:  2.5603084169505834
loss:  2.5602147197568113
loss:  2.5601211565945547
loss:  2.560027727169875
loss:  2.5599344311896632
loss:  2.5598412683616485
loss:  2.5597482383943837
loss:  2.5596553409972493
loss:  2.559562575880451
loss:  2.5594699427550096
loss:  2.55937744133277
loss:  2.5592850713263853
loss:  2.5591928324493245
loss:  2.559100724415865
loss:  2.559008746941089
loss:  2.55891689974088
loss:  2.5588251825319297
loss:  2.5587335950317196
loss:  2.5586421369585297
loss:  2.558550808031432
loss:  2.558459607970288
loss:  2.558368536495747
loss:  2.5582775933292425
loss:  2.558186778192986
loss:  2.558096090809974
loss:  2.558005530903974
loss:  2.55791509819953
loss:  2.5578

loss:  2.53580751650023
loss:  2.535745317271192
loss:  2.5356831888387066
loss:  2.535621131075885
loss:  2.535559143856143
loss:  2.5354972270532077
loss:  2.5354353805411085
loss:  2.535373604194185
loss:  2.5353118978870763
loss:  2.5352502614947308
loss:  2.5351886948923945
loss:  2.535127197955622
loss:  2.5350657705602626
loss:  2.5350044125824716
loss:  2.5349431238986995
loss:  2.534881904385699
loss:  2.5348207539205183
loss:  2.534759672380504
loss:  2.534698659643299
loss:  2.534637715586842
loss:  2.5345768400893633
loss:  2.5345160330293908
loss:  2.5344552942857437
loss:  2.534394623737534
loss:  2.5343340212641623
loss:  2.5342734867453247
loss:  2.5342130200610016
loss:  2.5341526210914656
loss:  2.534092289717275
loss:  2.534032025819278
loss:  2.5339718292786078
loss:  2.5339116999766818
loss:  2.533851637795206
loss:  2.533791642616165
loss:  2.5337317143218314
loss:  2.533671852794759
loss:  2.53361205791778
loss:  2.5335523295740128
loss:  2.5334926676468514
loss:

...

loss:  2.458950344633869
loss:  2.45894961081212
loss:  2.4589488773934414
loss:  2.4589481443775707
loss:  2.458947411764245
loss:  2.4589466795532045
loss:  2.4589459477441844
loss:  2.4589452163369225
loss:  2.4589444853311577
loss:  2.4589437547266284
loss:  2.4589430245230735
loss:  2.4589422947202295
loss:  2.4589415653178373
loss:  2.458940836315636
loss:  2.4589401077133646
loss:  2.4589393795107606
loss:  2.458938651707564
loss:  2.4589379243035174
loss:  2.4589371972983574
loss:  2.4589364706918255
loss:  2.4589357444836613
loss:  2.4589350186736048
loss:  2.458934293261398
loss:  2.4589335682467803
loss:  2.458932843629492
loss:  2.4589321194092757
loss:  2.4589313955858714
loss:  2.458930672159022
loss:  2.458929949128467
loss:  2.4589292264939484
loss:  2.4589285042552094
loss:  2.4589277824119926
loss:  2.4589270609640375
loss:  2.4589263399110877
loss:  2.4589256192528866
loss:  2.458924898989176
loss:  2.4589241791196996
loss:  2.4589234596441996
loss:  2.45892274056241

loss:  2.458678701037455
loss:  2.4586781132054125
loss:  2.4586775256813156
loss:  2.4586769384649636
loss:  2.4586763515561567
loss:  2.4586757649546946
loss:  2.458675178660376
loss:  2.458674592673003
loss:  2.4586740069923745
loss:  2.4586734216182893
loss:  2.458672836550551
loss:  2.4586722517889568
loss:  2.4586716673333093
loss:  2.458671083183409
loss:  2.458670499339056
loss:  2.4586699158000522
loss:  2.4586693325661977
loss:  2.4586687496372948
loss:  2.458668167013144
loss:  2.4586675846935475
loss:  2.458667002678306
loss:  2.4586664209672233
loss:  2.458665839560099
loss:  2.4586652584567363
loss:  2.458664677656938
loss:  2.458664097160505
loss:  2.4586635169672393
loss:  2.458662937076946
loss:  2.458662357489425
loss:  2.458661778204481
loss:  2.4586611992219174
loss:  2.4586606205415342
loss:  2.4586600421631375
loss:  2.45865946408653
loss:  2.458658886311514
loss:  2.458658308837895
loss:  2.4586577316654754
loss:  2.458657154794059
loss:  2.4586565782234495
loss:

loss:  2.4584604131681367
loss:  2.458459936648953
loss:  2.45845946036525
loss:  2.458458984316874
loss:  2.458458508503673
loss:  2.458458032925492
loss:  2.4584575575821797
loss:  2.458457082473582
loss:  2.458456607599548
loss:  2.458456132959923
loss:  2.458455658554558
loss:  2.458455184383295
loss:  2.4584547104459853
loss:  2.4584542367424778
loss:  2.458453763272618
loss:  2.458453290036254
loss:  2.4584528170332343
loss:  2.4584523442634083
loss:  2.458451871726622
loss:  2.458451399422726
loss:  2.4584509273515684
loss:  2.4584504555129962
loss:  2.45844998390686
loss:  2.4584495125330066
loss:  2.4584490413912863
loss:  2.4584485704815466
loss:  2.4584480998036384
loss:  2.4584476293574116
loss:  2.4584471591427115
loss:  2.4584466891593926
loss:  2.4584462194073002
loss:  2.4584457498862853
loss:  2.4584452805961985
loss:  2.458444811536887
loss:  2.458444342708204
loss:  2.458443874109998
loss:  2.4584434057421176
loss:  2.458442937604415
loss:  2.458442469696739
loss:  2

loss:  2.4582765120186147
loss:  2.458276123319712
loss:  2.4582757347991846
loss:  2.458275346456916
loss:  2.458274958292791
loss:  2.4582745703066924
loss:  2.4582741824985064
loss:  2.458273794868117
loss:  2.458273407415406
loss:  2.4582730201402607
loss:  2.4582726330425655
loss:  2.4582722461222053
loss:  2.458271859379064
loss:  2.458271472813026
loss:  2.4582710864239767
loss:  2.458270700211802
loss:  2.4582703141763864
loss:  2.458269928317614
loss:  2.4582695426353722
loss:  2.458269157129544
loss:  2.458268771800017
loss:  2.458268386646674
loss:  2.458268001669404
loss:  2.458267616868089
loss:  2.458267232242618
loss:  2.4582668477928724
loss:  2.4582664635187434
loss:  2.458266079420112
loss:  2.4582656954968676
loss:  2.458265311748895
loss:  2.45826492817608
loss:  2.4582645447783094
loss:  2.4582641615554692
loss:  2.4582637785074453
loss:  2.4582633956341255
loss:  2.4582630129353946
loss:  2.4582626304111397
loss:  2.4582622480612484
loss:  2.458261865885606
loss: 

loss:  2.4581364001124264
loss:  2.4581360733011084
loss:  2.45813574662798
loss:  2.4581354200929537
loss:  2.458135093695938
loss:  2.458134767436845
loss:  2.4581344413155843
loss:  2.458134115332067
loss:  2.4581337894862023
loss:  2.4581334637779038
loss:  2.458133138207079
loss:  2.458132812773641
loss:  2.4581324874774992
loss:  2.458132162318566
loss:  2.458131837296751
loss:  2.4581315124119665
loss:  2.458131187664123
loss:  2.4581308630531313
loss:  2.4581305385789025
loss:  2.4581302142413497
loss:  2.4581298900403823
loss:  2.458129565975912
loss:  2.4581292420478493
loss:  2.4581289182561092
loss:  2.4581285946005997
loss:  2.458128271081235
loss:  2.458127947697924
loss:  2.458127624450581
loss:  2.4581273013391156
loss:  2.4581269783634423
loss:  2.458126655523472
loss:  2.458126332819115
loss:  2.458126010250285
loss:  2.458125687816893
loss:  2.4581253655188524
loss:  2.458125043356075
loss:  2.4581247213284723
loss:  2.458124399435957
loss:  2.458124077678443
loss:  

In [ ]:
print(model2.w)

In [6]:
# get the rmse of the test data
print(model2.evaluate(test_data, test_labels))

1.7927420204914968


In [9]:
new_data=pd.DataFrame({
    'Country':["Australia"],
    'Year':[2013],
    'Status':[1],
    'Adult Mortality':[61],
    'infant deaths':[1],
    'Alcohol':[9.87],
    'percentage expenditure':[11734.85381],
    'Hepatitis B':[91],
    'Measles':[158],
    'BMI':[65.5],
    'under-five deaths':[1],
    'Polio':[91],
    'Total expenditure':[9.36],
    'Diphtheria':[91],
    'HIV/AIDS':[0.1],
    'GDP':[67792.3386],
    'Population':[23117353],
    'thinness  1-19 years':[0.6],
    'thinness 5-9 years':[0.6],
    'Income composition of resources':[0.933],
    'Schooling':[20.3]
})

new_data["Country_Australia"] = 1
new_data = new_data.drop(columns=["Country"])
train_data, new_data = train_data.align(new_data, join='outer', axis=1, fill_value=0)

print(model2.predict(new_data))

[83.45481149]
